In [1]:
import os
from dotenv import load_dotenv
from datasets import load_dataset
from smolagents import CodeAgent, LiteLLMModel
import yaml
from tools.final_answer import FinalAnswerTool
from tools.math_operations import MathOperationsTool
from tools.visit_webpage import VisitWebpageTool
from tools.web_search import DuckDuckGoSearchTool
from tools.wikipedia_search import WikipediaSearchTool
from tools.arxiv_search import ArxivSearchTool
from tools.rag_search import RAGSearchTool
from tools.code_execution import CodeExecutionTool
from tools.document_processing import DocumentProcessingTool
from tools.image_processing import ImageProcessingTool
from tools.web_scraping import WebScrapingTool

In [2]:
visit_webpage = VisitWebpageTool()
internet_search = DuckDuckGoSearchTool(max_results=5)
math_tools = MathOperationsTool()
final_answer = FinalAnswerTool()
wikipedia_search = WikipediaSearchTool(load_max_docs=2)
arxiv_search = ArxivSearchTool(load_max_docs=3)
rag_search = RAGSearchTool(persist_dir="rag_db")
code_execution = CodeExecutionTool()
document_processing = DocumentProcessingTool(temp_dir="doc_processing")
image_generation_tool = ImageProcessingTool()
web_scraping = WebScrapingTool()

c:\repo\courses\agents\Final_Project-HF_Agents_Course\HF_GAIA_AGENT\tools\rag_search.py:32: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embeddings = HuggingFaceEmbeddings(


In [ ]:
customs_tools = [    
    visit_webpage, 
    internet_search, 
    math_tools,
    wikipedia_search, 
    arxiv_search,
    rag_search,
    web_scraping,
    code_execution, 
    document_processing, 
    image_generation_tool,
    final_answer
]

additional_imports = [
    # Document processing
    "pymupdf",                              # PDF reading
    # HTTP & URLs
    "requests",                             # HTTP client for REST calls :contentReference[oaicite:0]{index=0}
    "urllib.parse",                         # URL parsing and construction :contentReference[oaicite:1]{index=1}

    # Data formats
    "json",                                 # JSON serialization/deserialization
    "csv",                                  # CSV reading/writing
    "xml.etree.ElementTree",                # XML parsing
    "bs4",                                  # BeautifulSoup for HTML parsing

    # Text processing
    "re",                                   # Regular expressions

    # File & OS
    "os",                                   # OS interactions (env vars, paths)
    "sys",                                  # Interpreter info
    "pathlib",                              # Object‑oriented filesystem paths
    "subprocess",                           # Safe subprocess calls

    # Computation
    "math",                                 # Advanced math functions
    "random",                               # Random sampling and shuffling

    # Date & time
    "datetime",                             # Date/time parsing and arithmetic

    # Data analysis
    "numpy",                                # Numerical arrays :contentReference[oaicite:2]{index=2}
    "pandas",                               # Tabular data manipulation

    # Imaging
    "PIL.Image",                            # Image loading/inspection

    # Logging
    "logging",                              # Structured debug/info logging    
]

In [4]:
# 3. Inicializar modelo Ollama local
local_model = LiteLLMModel(
    model_id="ollama/qwen2.5-coder:7b",    
    api_base="http://localhost:11434",
    temperature=0,
    max_tokens=2096,
    request_timeout=900,
)

In [5]:
# Cargar las plantillas de los prompts
with open("prompt.yaml", 'r') as stream:
    prompt_templates = yaml.safe_load(stream)

In [6]:
# Trace agent and send traces to Phoenix
from phoenix.otel import register
from openinference.instrumentation.smolagents import SmolagentsInstrumentor

register()
SmolagentsInstrumentor().instrument()

OpenTelemetry Tracing Details
|  Phoenix Project: default
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [6]:
# 4. Crear agente con el modelo local
agent = CodeAgent(
    model=local_model,
    tools = customs_tools, 
    additional_authorized_imports = additional_imports,
    prompt_templates=prompt_templates,
    #max_steps=5,
    verbosity_level = 1,
)

In [22]:
print(agent.prompt_templates)


{'system_prompt': 'You are an expert assistant with full access to following tools: {{ tools }}. \nThese tools are basically Python functions which you can call with code.\nTo solve the task, you must plan forward to proceed in a series of steps, in a cycle of \'Thought:\', \'Code:\', and \'Observation:\' sequences.\nAt each step, in the \'Thought:\' sequence, you should first explain your reasoning towards solving the task and the tools that you want to use.\nThen in the \'Code:\' sequence, you should write the code in simple Python. The code sequence must end with \'<end_code>\' sequence.\nDuring each intermediate step, you can use \'print()\' to save whatever important information you will then need.\nThese print outputs will then appear in the \'Observation:\' field, which will be available as input for the next step.\nIn the end you have to return a final answer using the `final_answer` tool. \n\nHere are the rules you should always follow to solve your task:\n1. Understand the ta

In [7]:
dataset_validation = load_dataset(
    "gaia-benchmark/GAIA",
    '2023_all',    
    split="validation",
    token=os.getenv("HF_TOKEN"),    
    trust_remote_code=True,
)

In [ ]:
#dataset_test = load_dataset(
#    "gaia-benchmark/GAIA",
#    '2023_all',    
#    split="test",
#    token=os.getenv("HF_TOKEN"),
#    trust_remote_code=True,
#)

In [8]:
print("---- Test set ----")
indices = list(range(1))
split_dataset_validation = dataset_validation.select(indices)
for example in split_dataset_validation:
    print(example)  

---- Test set ----
{'task_id': 'c61d22de-5f6c-4958-a7f6-5e9707bd3466', 'Question': 'A paper about AI regulation that was originally submitted to arXiv.org in June 2022 shows a figure with three axes, where each axis has a label word at both ends. Which of these words is used to describe a type of society in a Physics and Society article submitted to arXiv.org on August 11, 2016?', 'Level': '2', 'Final answer': 'egalitarian', 'file_name': '', 'file_path': '', 'Annotator Metadata': {'Steps': '1. Go to arxiv.org and navigate to the Advanced Search page.\n2. Enter "AI regulation" in the search box and select "All fields" from the dropdown.\n3. Enter 2022-06-01 and 2022-07-01 into the date inputs, select "Submission date (original)", and submit the search.\n4. Go through the search results to find the article that has a figure with three axes and labels on each end of the axes, titled "Fairness in Agreement With European Values: An Interdisciplinary Perspective on AI Regulation".\n5. Note t

In [ ]:
# 5. Ejecutar preguntas y recoger respuestas
predictions = []
for example in split_dataset_validation:
    q = example["Question"]
    print("Agent started running...")
    pred = agent.run(q)
    predictions.append(str(pred).strip())
    print("Agent finished running...")
    print(f"QUESTION: {q}\nANSWER: {pred}\n")

Agent started running...


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ A paper about AI regulation that was originally submitted to arXiv.org in June 2022 shows a figure with three   │
│ axes, where each axis has a label word at both ends. Which of these words is used to describe a type of society │
│ in a Physics and Society article submitted to arXiv.org on August 11, 2016?                                     │
│                                                                                                                 │
╰─ LiteLLMModel - ollama/qwen2.5-coder:7b ────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Step 1: Find the paper about AI regulation submitted in June 2022                                              
  ai_regulation_paper = arxiv_search.forward(query="AI regulation submission date:2022-06")                        
  print(ai_regulation_paper)                                                                                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Error searching arXiv: PyMuPDF package not found, please install it with `pip install pymupdf`

Out: None

...

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Install PyMuPDF package correctly                                                                              
  !pip install pymupdf                                                                                             
                                                                                                                   
  # Retry finding the paper about AI regulation submitted in June 2022                                             
  ai_regulation_paper = arxiv_search.forward(query="AI regulation submission date:2022-06")                        
  print(ai_regulation_paper)                                                                                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Error searching arXiv: PyMuPDF package not found, please install it with `pip install pymupdf`

Code parsing failed on line 2 due to: SyntaxError
!pip install pymupdf
 ^
Error: invalid syntax (<unknown>, line 2)

[Step 7: Duration 84.28 seconds| Input tokens: 22,013 | Output tokens: 751]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 8 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [24]:
# 6. Calcular exact‑match accuracy
gold = [ex["Final answer"].strip() for ex in split_dataset_validation]
correct = sum(1 for p, g in zip(predictions, gold) if p == g)
acc = correct / len(gold)
print(f"Exact‑match accuracy on GAIA dev: {acc:.2%}")

Exact‑match accuracy on GAIA dev: 0.00%
